In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

# Configuration

In [ ]:
# Configuration (environment)
#root_dir = '/home/christoph/Documents/SmartInverter/smartinverter_optimization'
#system = 'Linux64'
root_dir = r'C:\Users\Christoph\Documents\PrivateRepos\doper_private'
#root_dir = r'C:\Users\Christoph\Desktop'
#system = 'Windows64'
#system = 'Windows32'

# Configuration (optimization)
#solver = 'ipopt'

# Variables and Parameter
#solver_dir = os.path.join('SlowActing','Solvers')
solver_dir = 'solvers'
#solver_path = os.path.join(root_dir, solver_dir, system, solver)

# DOPER Package

### Example Module

In [ ]:
#from example import example_parameter_evfleet, example_inputs, example_inputs_evfleet2

### Utility Module

In [ ]:
from utility import pandas_to_dict, pyomo_read_parameter, get_solver, standard_report, resample_variable_ts

### Wrapper Module

In [ ]:
from wrapper import DOPER

### Basemodel Module

In [ ]:
#from basemodel import base_model, convert_base_model

### Batterymodel Module

In [ ]:
#from batterymodel import add_battery, convert_battery

### Batterydegradationmodule Module

In [ ]:
#from batterydegradationmodel import add_batterydegradation, convert_batterydegradation_model

### Controller Module

In [ ]:
#!/usr/bin/env python

'''
    INTERNAL USE ONLY
    Module of DOPER package (v1.0)
    cgehbauer@lbl.gov

    Version info (v1.0):
        -) Initial disaggregation of old code.
'''

import os
import sys
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from pyomo.environ import Objective, minimize

from wrapper import DOPER
from utility import get_solver, get_root, standard_report
from basemodel import base_model, convert_base_model, plot_standard1
from batterymodel import add_battery, convert_battery, plot_battery1
from example import example_parameter_evfleet, example_inputs_evfleet2

def control_model(inputs, parameter):
    model = base_model(inputs, parameter)
    model = add_battery(model, inputs, parameter)
    
    if 'weight_degradation' in parameter['objective']:
        print('WARNING: No "degradation" in objective function.')
    def objective_function(model):
        return model.sum_energy_cost * parameter['objective']['weight_energy'] \
               + model.sum_demand_cost * parameter['objective']['weight_demand'] \
               + model.sum_export_revenue * parameter['objective']['weight_export'] \
               + model.sum_regulation_revenue * parameter['objective']['weight_regulation']
    model.objective = Objective(rule=objective_function, sense=minimize, doc='objective function')
    return model
    
def pyomo_to_pandas(model, parameter):
    df = convert_base_model(model, parameter)
    df = pd.concat([df, convert_battery(model, parameter)], axis=1)
    return df 
    
if __name__ == '__main__':
    from computetariff import compute_periods
    
    parameter = example_parameter_evfleet()
    data = example_inputs_evfleet2(parameter)
    data.index = [ix.replace(month=6) for ix in data.index]
    del parameter['objective']['weight_degradation']
    
    # Testing
    #tariff = get_e19_2019_tariff()
    #data, parameter = compute_periods(data.copy(), tariff, parameter)

    smartDER = DOPER(model=control_model,
                     parameter=parameter,
                     solver_path=get_solver('cbc', solver_dir='solvers'),
                     pyomo_to_pandas=pyomo_to_pandas)
    res = smartDER.do_optimization(data, tee=False)
    duration, objective, df, model, result, termination, parameter = res
    print(standard_report(res))

In [ ]:
from computetariff import compute_periods

sys.path.append(r'C:\Users\Christoph\Documents\PrivateRepos\doper_private\examples\data')
from tariff import get_e19_2019_tariff

parameter = example_parameter_evfleet()
data = example_inputs_evfleet2(parameter)
data.index = [ix.replace(month=6) for ix in data.index]
del parameter['objective']['weight_degradation']

tariff = get_e19_2019_tariff()
data, parameter = compute_periods(data.copy(), tariff, parameter)

data = data.resample('1H').mean()
    
smartDER = DOPER(model=control_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir='solvers'),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data, tee=False)
duration, objective, df, model, result, termination, parameter = res
print(standard_report(res))

In [ ]:
plot_standard1(df)

### Regulation

In [ ]:
from computetariff import compute_periods

sys.path.append(r'C:\Users\Christoph\Documents\PrivateRepos\doper_private\examples\data')
from tariff import get_e19_2019_tariff

parameter = example_parameter_evfleet()
data = example_inputs_evfleet2(parameter)
data.index = [ix.replace(month=6) for ix in data.index]
del parameter['objective']['weight_degradation']

tariff = get_e19_2019_tariff()
data, parameter = compute_periods(data.copy(), tariff, parameter)

# Enable regulation
parameter['site']['regulation'] = True
parameter['site']['regulation_min'] = None
parameter['site']['regulation_symmetric'] = True
parameter['site']['regulation_all'] = False
parameter['site']['export_max'] = 1e6
parameter['objective']['weight_regulation'] = 22
data['tariff_regup'] = 0.005 # $/KWh
data['tariff_regdn'] = 0.004 # $/KWh
data = data.resample('1H').mean()

# Lowlevel
# parameter['site']['regulation'] = True
# parameter['site']['regulation_reserved'] = False
# parameter['site']['regulation_reserved_battery'] = False
# parameter['site']['regulation_reserved_variable_battery'] = True
# parameter['site']['regulation_symmetric'] = True
# data['regulation_up'] = df['Reg Up [kW]']
# data['regulation_dn'] = df['Reg Dn [kW]']
# for b in range(int(parameter['battery']['count'])):
#     data['battery_{}_regup'.format(b)] = df['Battery {} Reg Up [kW]'.format(b)]
#     data['battery_{}_regdn'.format(b)] = df['Battery {} Reg Dn [kW]'.format(b)]
    
smartDER = DOPER(model=control_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir='solvers'),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data, tee=False)
duration, objective, df, model, result, termination, parameter = res
print(standard_report(res))

In [ ]:
plot_standard1(df)

In [ ]:
df

In [ ]:
df['NetReg'] = df['Reg Dn [kW]']
df[['Battery 0 Power [kW]','Battery 1 Power [kW]','Battery 2 Power [kW]',
    'Battery 0 AC Power [kW]','Battery 1 AC Power [kW]','Battery 2 AC Power [kW]',
    'Battery AC Power [kW]','NetReg']].round(0)

In [ ]:
df['NetLoad'] = df['Load Power [kW]'] + df['PV Power [kW]']
df['NetLoadBatt'] = df['NetLoad'] + df['Battery AC Power [kW]']

df[['Import Power [kW]','NetLoad','NetLoadBatt']].plot()

In [ ]:
plot_standard1(df)

### Variable Timestep

In [ ]:
from computetariff import compute_periods

sys.path.append(r'C:\Users\Christoph\Documents\PrivateRepos\doper_private\examples\data')
from tariff import get_e19_2019_tariff

parameter = example_parameter_evfleet()
data = example_inputs_evfleet2(parameter)
data.index = [ix.replace(month=6) for ix in data.index]
del parameter['objective']['weight_degradation']

data = resample_variable_ts(data, reduced_start=60, reduced_ts=15)

tariff = get_e19_2019_tariff()
data, parameter = compute_periods(data.copy(), tariff, parameter)
    
smartDER = DOPER(model=control_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir='solvers'),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data, tee=False)
duration, objective, df, model, result, termination, parameter = res
print(standard_report(res))

In [ ]:
#from controller import control_model, pyomo_to_pandas, plot_standard1, plot_battery1

In [ ]:
# TODO
# Filter battery issues in advance (avoid infeasibility); or slack variables
# First controller as charging manager
# Penalize movement
# Todo include end of life (otherwise degradation * cost)
# Cycle aging
# RC parameter
# Better implementation of degradation
# Temperature and degradation also during driving?
# Which scenarios? Regulation and V2B at same time or XOR; Hard in reality, easy in simulation

# Done - Split power battery, external power battery (where to add availability and external demand; battery SOC calculation)
# Done - Battery demand when unplugged
# Done - Split objective and total energy cost (for penalty)
# Done - Battery thermal model
# Done - Battery degradation model

In [ ]:
#parameter = dafault_parameter()
parameter = example_parameter_evfleet()

#data = example_inputs(parameter)
data = example_inputs_evfleet2(parameter)

# Change weights for testing
parameter['objective']['weight_export'] = 0 # Weight of revenue (export) in objective
parameter['objective']['weight_regulation'] = 0 # Weight of revenue (regulation) in objective
parameter['objective']['weight_degradation'] = 30 # Weight of battery degradation cost in objective
#del parameter['objective']['weight_degradation']

smartDER = DOPER(model=control_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data, tee=False)
duration, objective, df, model, result, termination, parameter = res
print(standard_report(res))

In [ ]:
plot_standard1(df)

In [ ]:
plot_battery1(df, model)

### CEC CPR Report

In [ ]:
print(STOP)

In [ ]:
parameter['objective']['weight_degradation'] = 0
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data)
duration, objective, df, model, result, termination = res
print (standard_report(res))

In [ ]:
parameter['objective']['weight_degradation'] = 0
parameter['battery']['soc_initial'] = [0] * 3
parameter['battery']['soc_final'] = [0] * 3
parameter['battery']['self_discharging'] = [0] * 3 
for b in range(parameter['battery']['count']):
    data['battery_{!s}_avail'.format(b)] = 0
    data['battery_{!s}_demand'.format(b)] = 0
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data)
duration, objective, df, model, result, termination = res
print (standard_report(res))

In [ ]:
parameter['objective']['weight_degradation'] = 1
parameter['battery']['soc_initial'] = [0] * 3
parameter['battery']['soc_final'] = [0] * 3
parameter['battery']['self_discharging'] = [0] * 3 
for b in range(parameter['battery']['count']):
    data['battery_{!s}_avail'.format(b)] = 0
    data['battery_{!s}_demand'.format(b)] = 0
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data)
duration, objective, df, model, result, termination = res
print (standard_report(res))

In [ ]:
parameter['objective']['weight_degradation'] = 1
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data)
duration, objective, df, model, result, termination = res
print (standard_report(res))

# Test with different solver

In [ ]:
solvers = ['cbc','ipopt','couenne']
for solver in solvers:
    smartDER = DOPER(model=SmartInverter_model,
                     parameter=parameter,
                     solver_path=get_solver(solver, solver_dir=solver_dir),
                     pyomo_to_pandas=pyomo_to_pandas)
    res = smartDER.do_optimization(data)
    duration, objective, df, model, result, termination = res
    print (standard_report(res))

In [ ]:
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_pathget_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data.iloc[0:12])
duration, objective, df, model, result, termination = res
plot_standard1(df, plot_times=False)

In [ ]:
#result.write()

In [ ]:
model.pprint()

# Test new Optimization module (with variable timestep)

In [ ]:
# Example INPUT
data = pd.read_csv(os.path.join(root_dir, 'ExampleData', 'Flexlab.csv'))
data.index = pd.to_datetime(data['Date/Time'].apply(lambda x: '2018/'+x[1:6]+' '+'{:2d}'.format(int(x[8:10])-1)+x[10::]))
del data.index.name
data = data[['FLEXLAB-X3-ZONEA:Zone Air Heat Balance System Air Transfer Rate [W](Hourly)']]
data.columns = ['load_demand']
data['load_demand'] = data['load_demand'].mask(data['load_demand']<0, data['load_demand']*-1/3.5) / 1000.
# Use only 1 day
data = data.iloc[0:24]
# Scale Load data
data['load_demand'] = data['load_demand'] * 4
data['tariff_energy_map'] = 0
data['tariff_energy_map'] = data['tariff_energy_map'].mask((data.index.hour>=8) & (data.index.hour<22), 1)
data['tariff_energy_map'] = data['tariff_energy_map'].mask((data.index.hour>=12) & (data.index.hour<18), 2)
data['tariff_power_map'] = data['tariff_energy_map'] # Apply same periods to demand charge
data['tariff_energy_export_map'] = 0
data['generation_pv'] = 0
data['generation_pv'].iloc[10:16] = [np.sin(i/(5/(np.pi))) * 4 for i in range(6)]
data['tariff_regup'] = data['tariff_power_map'] * 0.05 + 0.01
data['tariff_regdn'] = data['tariff_power_map'] * 0.01 + 0.01
data['date_time'] = data.index
#data = data.reset_index(drop=True)
#data

# Test the Script

In [ ]:
#from SlowActingControlv10 import do_optimization, plot_standard1

parameter['site']['regulation'] = False
parameter['battery']['soc_initial'] = 0
parameter['battery']['soc_final'] = 0
parameter['battery']['soc_min'] = 0
parameter['battery']['soc_max'] = 1

In [ ]:
print ('Resampled to 5 minute (no Regulation)')
data_rs = data.resample('5T').ffill()
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data_rs.loc['2018-01-01 00:00:00':'2018-01-01 22:55:00'])
duration, objective, df, model, result, termination = res
print (standard_report(res))
plot_standard1(df)

In [ ]:
print ('Basecase with hourly timestep (no Regulation)')
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data.loc['2018-01-01 00:00:00':'2018-01-01 22:00:00'])
duration, objective, df, model, result, termination = res
print (standard_report(res))
plot_standard1(df)

### Test with DefaultConfiguration function

In [ ]:
from DefaultConfiguration import get_parameter_slowacting_flexgrid, get_input_example2

parameter = get_parameter_slowacting_flexgrid()
data = get_input_example2()

print ('Resampled to 5 minute (no Regulation)')
data_rs = data.resample('5T').ffill()
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data_rs.loc['2018-01-01 00:00:00':'2018-01-01 22:55:00'])
duration, objective, df, model, result, termination = res
print (standard_report(res))
plot_standard1(df)

In [ ]:
get_parameter_slowacting_flexgrid()

# Test Major and Minor

In [ ]:
parameter = get_parameter_slowacting_flexgrid()
data = get_input_example2()
print ('Running Major')
parameter['site']['regulation'] = True
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data)
duration, objective, df, model, result, termination = res
print (standard_report(res))
df[['Reg Up [kW]','Reg Dn [kW]']].plot(figsize=(16,3))
plt.show()
#plot_standard1(df)

print ('Running Minor (hourly)')
data['battery_reg'] = (-1*df['Reg Up [kW]'] + df['Reg Dn [kW]']) * 0.95
#print data['battery_reg']
parameter['site']['regulation'] = False
parameter['site']['regulation_reserved'] = True
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data)
duration, objective, df, model, result, termination = res
print (standard_report(res))
df[['Reg Up [kW]','Reg Dn [kW]']].plot(figsize=(16,3))
plt.show()
#plot_standard1(df)


print ('Running Minor (5-min)')
parameter['site']['regulation'] = False
parameter['site']['regulation_reserved'] = True
data_rs = data.resample('5T').ffill()
smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data_rs)
duration, objective, df, model, result, termination = res
print (standard_report(res))
df[['Reg Up [kW]','Reg Dn [kW]']].plot(figsize=(16,3))
plt.show()
#plot_standard1(df)

# Test battery self-discharge

In [ ]:
fig, ax1 = plt.subplots(figsize = (12,3))
ax2 = ax1.twinx()
p = ax1.plot(df['Battery Energy [kWh]'], 'g-')
p += ax2.plot(df['Battery Self Discharge [kW]'], 'b-')
plt.legend(p, [l.get_label() for l in p])
plt.show()

print ('Self-discharge @ 6.4 kWh with 0.3 %/24h = 0.08 kW')

# RC Thermal Model

In [ ]:
parameter = dafault_parameter()
data = example_inputs()

# Fix Outside tmeperature
data['oat'] = 22

smartDER = DOPER(model=SmartInverter_model,
                 parameter=parameter,
                 solver_path=get_solver('cbc', solver_dir=solver_dir),
                 pyomo_to_pandas=pyomo_to_pandas)
res = smartDER.do_optimization(data)
duration, objective, df, model, result, termination = res
#print (standard_report(res))

df[['Battery 1 Power [kW]']].plot(figsize=(12,2))
df[['Battery 1 Temperature [C]']].plot(figsize=(12,2))
plt.plot()

In [ ]:
# Refer to LAAFB/optimization/RC Model.ipynb
print('Battery loss: {:.0f} W (should be 131 W)'.format(df[['Battery 1 Power [kW]']] \
                                                 .loc['2018-01-01 01:00':'2018-01-01 01:55'].mean().values[0] \
                                                 * (1-parameter['battery']['efficiency_charging'][0]) * 1000))
predicted = 22.6 # Refer to RC Model.ipynb
measured = round(df[['Battery 1 Temperature [C]']].loc['2018-01-01 01:55'].values[0], 1)
print('Battery Temperature predicted is {!s} C and MPC is {!s}'.format(predicted, measured))
print('Passed? {!s}'.format(predicted==measured))

# Battery Degradation Model (Wang)

In [ ]:
from BatteryDegradationModel import Wang_model

In [ ]:
#Wang_model(T, C, E, days, mode='Wang', temp_range=[0,50])
Wang_model(10, 0, 0, 500)